In [1]:
## Categorize and write CSV of Tweets Data
import pandas as pd
import time
import os
import numpy as np
path = 'json/'
files = os.listdir(path)
all_files = []
# print files
for f in files:
    if f.endswith(".json"):
        all_files.append(f)
count_files = len(all_files)
print "Total file number is: " + str(count_files)
times = count_files/280/3
print "Need Loop for almost " + str(times) + " times." + "\n"

Total file number is: 23459
Need Loop for almost 27 times.



In [2]:
prayer_dict = {'pray':'صلى','prayer':'صلاة','praying':'صلاة','muslim':'مسلم','Islam':'الإسلام',
               'mosque':'جامع','mosques':'جوامع','masjid':'مساجد','masjids':'مسجد',
               'call to prayer':'اذان','muezzin':'مؤذن','holy':'مقدس',
              'imam':'أئمة','imams':'امام','koran':'القرآن','chapter':'سور','verse':'آيات'}
food_dict = {'food':'طعام','eating':'يتناول الطعام','falafel':'فلافل','shawarma':'الشاورما',
            'starbucks':'ستاربكس','mcdonalds':'ماكدونالدز'}
fun_dict = {'restaurant':'مطاعم','restaurants':'مطعم','park':'حدائق','theater':'سينمات','museum':'متاحف',
           'bakery':'مخابز','gallery':'معرض','fair':'مهرجانات','zoo':'حديقة الحيوان','hotel':'فنادق',
           'starbucks':'ستاربكس','mcdonalds':'ماكدونالدز'}
parking_dict = {'stop':'يوقف','park':'وقف','parking lot':'موقف','parking':'مواقف','standing':'المواقف',
               'street parking':'يركن','garage':'يجرّش'}
traffic_dict = {'traffic':'مرور','underground':'سرا','crowd':'زحمة','crowds':'زحام','jammed':'اختناق',
                'erupted':'نشبت','reroute':'تحويلة','bump':'مطب','pothole':'مطبات'}
accident_dict = {'accident':'حادث','accidents':'حوادث','crashes':'حادثة','collision':'صدم','hit by':'صدمني',
                 'detour':'انعفطت',
                'wreck':'خربت','wreckage':'حطام'}
transit_dict = {'bus':'باص','taxi':'تاكسي','taxis':'تاكسيات','cab':'تكسي','limo':'ليموزين','fare':'أجرة',
                'cab fee':'اجرة','Careem':'كريم','Uber':'اوبر','driver':'سواق','chauffeur':'سائق','airport':'مطارات',
                'public transit':'مواصلات'}
# Search for Arabic terms using unicode
# It works on English too

def count_search_terms(arabic_word):
    j=0
    search_term = unicode(arabic_word, encoding="utf-8")
    for row in all_tweets.itertuples():
        if search_term.lower() in row[1].lower():
            j+=1
    return j
# Now count multiple terms at once
# The function counts both Arabic and English versions

def count_category(search_dict):
    i=0
    for key in search_dict:
        count = count_search_terms(search_dict[key])
        count = count + count_search_terms(key)
        i+=count
        ##print key + ": " + str(count)
    ##print "category total: " + str(i)
        
# function decides if category is present
def has_category(tweet_content, search_dict):
    answer = False
#     tweet_content = unicode(tweet_content, encoding="utf-8")
    for english_word in search_dict.keys():
        if english_word.lower() in tweet_content.lower():
            answer = True
    for arabic_word in search_dict.values():
        arabic_word = unicode(arabic_word, encoding="utf-8")
        if arabic_word in tweet_content:
            answer = True
    return answer

# function checks every row and adds categories
def fill_in_categories():
    for tweet in all_tweets.itertuples():
        idx = tweet[0]
        content = all_tweets.loc[idx,'content']
        if has_category(content, traffic_dict):
            all_tweets.loc[idx, 'category'] = 'traffic'
        elif has_category(content, transit_dict):
            all_tweets.loc[idx, 'category'] = 'transit'
        elif has_category(content, accident_dict):
            all_tweets.loc[idx, 'category'] = 'accident'
        elif has_category(content, parking_dict):
            all_tweets.loc[idx, 'category'] = 'parking'
        elif has_category(content, prayer_dict):
            all_tweets.loc[idx, 'category'] = 'prayer'
        elif has_category(content, fun_dict):
            all_tweets.loc[idx, 'category'] = 'fun'
        else:
            all_tweets.loc[idx, 'category'] = 'none' 
            
# function convert time format
def time_converter(input_string):
    ##orig_date = input_string[:-10] + input_string[-4:]
    ##convert = time.strptime(orig_date, '%a %b %d %H:%M:%S %Y')
    ##new_time = time.strftime("%A %I %p",convert)
    new_date = str(input_string[4:10])
    new_hour = str(input_string[11:13])
    return new_date, new_hour

### for loop to print the csv file
for m in range (20,28):
    x= 280*3*m
    y= x+280*3
    print m
    print (x,y)
    df = pd.DataFrame()
    for filename in all_files[x:y]:
        ##print filename
        new_path = "json"+"/" + str(filename)
        df0 = pd.read_json(new_path)
        df1 = df0.T
        df = df.append(df1)
    # reverse the columns and rows
    all_tweets = df
    print all_tweets.shape
    # Add and drop columns, reset index
    all_tweets['category'] = ""
    all_tweets.reset_index(drop=True, inplace=True)
    all_tweets.drop(['data_point','raw_source','tweet_id', 'user','user_location'],axis=1, inplace=True)
    ## Count Categories
    count_category(traffic_dict)
    count_category(parking_dict)
    count_category(accident_dict)
    count_category(transit_dict)
    count_category(prayer_dict)
    count_category(fun_dict)
    ## Fill in Categories
    fill_in_categories()
    
## Filter to choose certain category
    ###all_tweets_categorized = all_tweets[all_tweets['category']!= 'none']
    ##df = all_tweets[all_tweets['category']== 'prayer']    
    ## Change Time Format
    df = all_tweets
    for row in df.itertuples():
        idx = int(row[0])
        new_time = time_converter(df.loc[idx,'time'])
        df.loc[idx,'Date'] = str(new_time[0])
        df.loc[idx,'Hour'] = float(new_time[1])
    # df.to_csv("df_"+str(x)+"_"+str(y)+".csv", index=False, encoding='utf-8')
    # print "Done with Writing: "+"df_"+str(x)+"_"+str(y)+".csv"
    from collections import Counter
    Counter(df['Date'])
    df_new = pd.DataFrame()
    df_new['Hour'] = ""
    j=0
    for i in dict.keys(Counter(df['Date'])):
        for j in range(0,24):
            ##print i + ", " + "Hour: " +str(j)
            count= np.sum((df['Date']==i) & (df['Hour']== j))
            ##print count
            df_new.loc[j,i] = float(count)
            df_new.loc[j, 'Hour'] = j
    # df_new.head(5)
    df_new.to_csv("all_tweets/"+"df_new_"+str(x)+"_"+str(y)+".csv", index=False, encoding='utf-8')
    print "Done Writing: "+"df_new_"+str(x)+"_"+str(y)+".csv"

20
(16800, 17640)
(48672, 9)
Done Writing: df_new_16800_17640.csv
21
(17640, 18480)
(53151, 9)
Done Writing: df_new_17640_18480.csv
22
(18480, 19320)
(55505, 9)
Done Writing: df_new_18480_19320.csv
23
(19320, 20160)
(51616, 9)
Done Writing: df_new_19320_20160.csv
24
(20160, 21000)
(56914, 9)
Done Writing: df_new_20160_21000.csv
25
(21000, 21840)
(55922, 9)
Done Writing: df_new_21000_21840.csv
26
(21840, 22680)
(53597, 9)
Done Writing: df_new_21840_22680.csv
27
(22680, 23520)
(51766, 9)
Done Writing: df_new_22680_23520.csv
